In [6]:
from cellfinder.core.main import main as cellfinder_run
import tifffile

signal_array = tifffile.imread("tests/data/signal/signal_ch000000.tif")
background_array = tifffile.imread("tests/data/background/background_ch010000.tif")

voxel_sizes = [5, 2, 2] # in microns
detected_cells = cellfinder_run(signal_array,background_array,voxel_sizes)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/srutanik/cellfinder-brainglobe/cellfinder/core/classify/tests/data/signal/signal_ch000000.tif'